In [1]:
import pandas as pd
import os
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [3]:
baojiadan_folder = 'D:\\Program Files (x86)\\百度云同步盘\\Dropbox\\-E·J- 2014.5.1\\2015.9.26 燕文\\'

In [4]:
baojiadan_file_list = list()

In [5]:
for file_name in os.listdir(baojiadan_folder):
    if file_name.startswith('广州燕文WISH报价单'):
        baojiadan_file_list.append(file_name)

In [6]:
latest_baojiadan_file_name = baojiadan_file_list[-1]
# latest_baojiadan_file_name

In [7]:
latest_baojiadan_path = os.path.join(baojiadan_folder, latest_baojiadan_file_name)
# latest_baojiadan_path

In [8]:
baojiadan_sheets_dict = pd.read_excel(latest_baojiadan_path, sheetname=None)
# type(baojiadan)

In [9]:
wanted_shipping_service_list = ['Wish燕文航空经济小包-普货',
                                'Wish燕文航空经济小包-特货',
                                'Wish燕文专线平邮小包-普货',
                                'Wish燕文专线平邮小包-特货',
                                'WISH燕文C平邮小包',
                                'Wish燕邮宝平邮-特货',
                                'Wish燕文航空挂号小包-普货',
                                'Wish燕文航空挂号小包-特货',
                                'WISH燕文C挂号小包',
                                'WISH燕文C优先小包',
                                'Wish燕邮宝挂号-特货',
                                'Wish燕文航空易派小包-普货',
                                'Wish燕文航空易派小包-特货',
                                'Wish燕文专线追踪小包-普货',
                                'Wish燕文专线追踪小包-特货',
                                'WISH燕特快-普货',
                                'WISH燕特快-特货'
]

In [10]:
export_path = os.path.join(baojiadan_folder, '报价单','燕文（wish）报价表格.xlsx')
writer = pd.ExcelWriter(export_path)

for service_name in wanted_shipping_service_list:
    if service_name in baojiadan_sheets_dict.keys():
        logging.info(service_name + ' is ready to export.')
        df = baojiadan_sheets_dict[service_name]
        try:
            for c in df.columns:
                for index, r in enumerate(df[c]):
                    if r == '国家':
                        country_cell_location = [index, c]
#                         print(country_cell_location)
                    if r == '价格使用说明:':
                        end_location = [index, c]
#                         print(country_cell_location)
                        break
            df = df[country_cell_location[0]+1:end_location[0]]
            df = df.dropna(how='all')
            df = df[df[country_cell_location[1]].str.len() < 10]
#             df['service_name'] = service_name
            print(df.head(5))
            
#             tiers = [1, 2, 3]
#             for tier in tiers:
#                 this_c_name = 'tier_' + str(tier)
#                 next_c = tier + 1
#                 try:
#                     df[this_c_name + '_base'] = df[[tier]]
#                     df[this_c_name + '_surcharge'] = df[[next_c]]
#                     df[this_c_name + '_total'] = df[[tier]].astype(float) * 0.35 + df[this_c_name + '_surcharge'].astype(float)
#                 except Exception as err:
#                     print(str(err))
            
            kg = 0.35
            tiers = [1, 2, 3]
            
            if service_name == 'Wish燕文航空易派小包-普货':
                tiers = [1]
            
            for tier in tiers:
                this_c_name = 'tier_' + str(tier)
                column_num = 2 * tier - 1
                try:
                    df[this_c_name + '_kg_fee'] = df[[column_num]] * kg
                    df[this_c_name + '_kg_fee'] = df[this_c_name + '_kg_fee'].astype(float)
                    df[this_c_name + '_surcharge'] = df[[column_num + 1]].astype(float)
                    df[this_c_name + '_total'] = df[this_c_name + '_kg_fee'] + df[this_c_name + '_surcharge']
                except Exception as err:
                    print(str(err))
                    
#             df_for_merge = df[['service_name', 0, 'tier_1_total']]
            
#             pd_to_merge.append(df_for_merge)
    
            df.to_excel(writer, sheet_name=service_name, index=False)
            logging.info(service_name + ' exported.')
            # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.csv'))
        except Exception as err:
            logging.info(service_name + ' failed to export. ' + str(err))
            pass
        # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.xlsx'))
        
writer.save()

 2020-07-22 00:02:37,579 - INFO - Wish燕文航空经济小包-普货 is ready to export.
 2020-07-22 00:02:37,678 - INFO - Wish燕文航空经济小包-普货 exported.
 2020-07-22 00:02:37,678 - INFO - Wish燕文航空经济小包-特货 is ready to export.
 2020-07-22 00:02:37,784 - INFO - Wish燕文航空经济小包-特货 exported.
 2020-07-22 00:02:37,785 - INFO - Wish燕文专线平邮小包-普货 is ready to export.
 2020-07-22 00:02:37,811 - INFO - Wish燕文专线平邮小包-普货 exported.
 2020-07-22 00:02:37,812 - INFO - Wish燕文专线平邮小包-特货 is ready to export.
 2020-07-22 00:02:37,837 - INFO - Wish燕文专线平邮小包-特货 exported.


  Wish燕文航空经济小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  返回主目录
4              美国        110         13        110         13    NaN
5              日本         52        7.3        115       0.65    NaN
6              英国         65          9         65          9    NaN
7              法国         76          9         76          9    NaN
8              德国         80          9         80          9    NaN
  Wish燕文航空经济小包-特货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  返回主目录
4              美国        110         13        110         13    NaN
5              日本         58          9        115       0.65    NaN
6              英国         65          9         65          9    NaN
7              法国         76          9         76          9    NaN
8              德国         95          9         95          9    NaN
  Wish燕文专线平邮小包-普货 Unnamed: 1 Unnamed: 2      Unnamed: 3 Unnamed: 4  返回主目录
3              英国       57.5        8.9  0.001-0.149 KG      0.001    NaN
4              英国       

 2020-07-22 00:02:37,838 - INFO - Wish燕邮宝平邮-特货 is ready to export.
 2020-07-22 00:02:37,871 - INFO - Wish燕邮宝平邮-特货 exported.
 2020-07-22 00:02:37,872 - INFO - Wish燕文航空挂号小包-普货 is ready to export.
 2020-07-22 00:02:37,988 - INFO - Wish燕文航空挂号小包-普货 exported.
 2020-07-22 00:02:37,989 - INFO - Wish燕文航空挂号小包-特货 is ready to export.


  Wish燕邮宝平邮-特货 Unnamed: 1 Unnamed: 2    Unnamed: 3 Unnamed: 4  返回主目录
3           英国       45.5      10.35  0.001-2.0 KG       0.01    NaN
4           法国       46.5       9.35  0.001-2.0 KG       0.01    NaN
5           德国       48.5       9.35  0.001-2.0 KG       0.01    NaN
6          意大利       41.5      22.35  0.001-2.0 KG       0.01    NaN
7          西班牙       53.5      10.35  0.001-2.0 KG       0.01    NaN
Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
  Wish燕文航空挂号小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4              美国        100         25        100         25        100   
5              日本         55         15         55         15         55   
6              英国         48         17         48         17         48   
7              法国         75         17         75         17         75   
8              德国         65         17         65         17         65   

  Unnamed: 6  返回主目录  
4         25   

 2020-07-22 00:02:38,079 - INFO - Wish燕文航空挂号小包-特货 exported.
 2020-07-22 00:02:38,080 - INFO - WISH燕文C挂号小包 is ready to export.
 2020-07-22 00:02:38,207 - INFO - WISH燕文C挂号小包 exported.
 2020-07-22 00:02:38,208 - INFO - WISH燕文C优先小包 is ready to export.
 2020-07-22 00:02:38,245 - INFO - WISH燕文C优先小包 exported.
 2020-07-22 00:02:38,246 - INFO - Wish燕邮宝挂号-特货 is ready to export.
 2020-07-22 00:02:38,279 - INFO - Wish燕邮宝挂号-特货 exported.
 2020-07-22 00:02:38,281 - INFO - Wish燕文航空易派小包-普货 is ready to export.


  WISH燕文C挂号小包 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4          美国        100         25        100         25        100   
5          日本         55         15         55         15         55   
6          英国         48         17         48         17         48   
7          法国         75         17         75         17         75   
8          德国         65         17         65         17         65   

  Unnamed: 6  返回主目录  
4         25    NaN  
5         15    NaN  
6         17    NaN  
7         17    NaN  
8         17    NaN  
  WISH燕文C优先小包 Unnamed: 1 Unnamed: 2    Unnamed: 3 Unnamed: 4  返回主目录
3          美国        100         25  0.001-0.3 KG       0.05    NaN
4          美国        100         25  0.301-2.0 KG       0.05    NaN
5          日本         55         15  0.001-0.3 KG       0.05    NaN
6          日本         55         15  0.301-2.0 KG       0.05    NaN
7          英国         48         17  0.001-2.0 KG       0.05    NaN
Could not operate 0.35 with

 2020-07-22 00:02:38,344 - INFO - Wish燕文航空易派小包-普货 exported.
 2020-07-22 00:02:38,345 - INFO - Wish燕文航空易派小包-特货 is ready to export.
 2020-07-22 00:02:38,412 - INFO - Wish燕文航空易派小包-特货 exported.
 2020-07-22 00:02:38,413 - INFO - Wish燕文专线追踪小包-普货 is ready to export.
 2020-07-22 00:02:38,439 - INFO - Wish燕文专线追踪小包-普货 exported.
 2020-07-22 00:02:38,440 - INFO - Wish燕文专线追踪小包-特货 is ready to export.
 2020-07-22 00:02:38,467 - INFO - Wish燕文专线追踪小包-特货 exported.
 2020-07-22 00:02:38,468 - INFO - WISH燕特快-普货 is ready to export.
 2020-07-22 00:02:38,501 - INFO - WISH燕特快-普货 exported.
 2020-07-22 00:02:38,501 - INFO - WISH燕特快-特货 is ready to export.
 2020-07-22 00:02:38,524 - INFO - WISH燕特快-特货 exported.


  Wish燕文航空易派小包-特货 Unnamed: 1 Unnamed: 2  返回主目录
4              美国        107         28    NaN
5              日本         60         17    NaN
6              法国         75         17    NaN
7              德国         55         23    NaN
8             意大利         70         25    NaN
  Wish燕文专线追踪小包-普货 Unnamed: 1 Unnamed: 2    Unnamed: 3 Unnamed: 4  返回主目录
3              美国         95         16  0.001-0.2 KG       0.05    NaN
4              美国         90         15  0.201-0.4 KG       0.05    NaN
5              美国         90         15  0.401-0.7 KG       0.05    NaN
6              美国         85          9  0.701-1.5 KG       0.05    NaN
7              美国         85          9  1.501-2.0 KG       0.05    NaN
Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
  Wish燕文专线追踪小包-特货 Unnamed: 1 Unnamed: 2    Unnamed: 3 Unnamed: 4  返回主目录
3              美国        100       16.5  0.001-0.2 KG       0.05    NaN
4              美国         95         16  0.201-0.4